In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path
import math

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from utils import general_utils
from utils import eval_utils
import deformation.losses as def_losses

In [3]:
def compare_before_after_refinement(results_df_before, results_df_after):
    
    statistics = {"num_3d_iou_improved":0, "avg_improve_amt": 0, "avg_improve_before_2d_iou":0, "num_3d_iou_worse":0,  "avg_worse_amt":0, "avg_worse_before_2d_iou":0}
    improved_instances = []
    worse_instances = []
    comparision_df = pd.DataFrame()
    for instance in list(results_df_after["instance"]):
        if instance in list(results_df_before["instance"]):
            before_result = results_df_before[results_df_before["instance"]==instance]
            before_3d_iou = float(before_result["3d_iou"])
            #before_2d_iou = float(before_result["2d_iou"])
            after_result = results_df_after[results_df_after["instance"]==instance]
            after_3d_iou = float(after_result["3d_iou"])
            #after_2d_iou = float(after_result["2d_iou"])

            #iou_2d_change = after_2d_iou - before_2d_iou
            iou_3d_change = after_3d_iou - before_3d_iou

            if math.isnan(iou_3d_change):
                statistics["num_3d_iou_worse"] += 1
                continue
            elif iou_3d_change > 0:
                statistics["num_3d_iou_improved"] += 1
                statistics["avg_improve_amt"] += iou_3d_change
                #statistics["avg_improve_before_2d_iou"] += before_2d_iou
                improved_instances.append(instance)
            else:
                statistics["num_3d_iou_worse"] += 1
                statistics["avg_worse_amt"] += iou_3d_change
                #statistics["avg_worse_before_2d_iou"] += before_2d_iou
                worse_instances.append(instance)
            comparision_df = comparision_df.append({"instance": instance, "iou_3d_delta": iou_3d_change}, ignore_index=True)

    statistics["avg_improve_amt"] /= statistics["num_3d_iou_improved"]
    statistics["avg_improve_before_2d_iou"] /= statistics["num_3d_iou_improved"]
    statistics["avg_worse_amt"] /= statistics["num_3d_iou_worse"]
    statistics["avg_worse_before_2d_iou"] /= statistics["num_3d_iou_worse"]
            
    pprint.pprint(statistics)
    #print("example improved instances")
    #print(improved_instances[:100])
    #print("example worse instances")
    #print(worse_instances[:100])
    return comparision_df

In [4]:
def filter_eval_df_warnings(original_df, warnings_to_filter=["NaN nodes", "compute_iou_3d: gt mesh < 200 occupancies", "compute_iou_3d: rec mesh < 200 occupancies"]):
    filter_array = []
    warnings_list = original_df["eval_warnings"].tolist()
    for i in range(len(original_df)):
        warnings = warnings_list[i]
        keep=True
        for warning in warnings:
            if warning in warnings_to_filter:
                keep=False
        filter_array.append(keep)
    return original_df[filter_array]

def filter_and_sort_eval_dfs(df1, df2):
    df1 = filter_eval_df_warnings(df1)
    df2 = filter_eval_df_warnings(df2)
    df1_instances = df1["instance"].tolist()
    df2_instances = df2["instance"].tolist()
    
    df1_filters = [(True if df1_instance in df2_instances else False) for df1_instance in df1_instances]
    df1_filtered = df1[df1_filters].sort_values(by="instance")
    
    df2_filters = [(True if df2_instance in df1_instances else False) for df2_instance in df2_instances]
    df2_filtered = df2[df2_filters].sort_values(by="instance")
    
    return df1_filtered, df2_filtered

def update_2d_iou_input_to_bf(original_df, bf_2d_iou_df):
    if (original_df["instance"].tolist() != bf_2d_iou_df["instance"].tolist() ):
        raise ValueError("Instances don't match")
    original_df["2d_iou_input"] = bf_2d_iou_df["2d_iou_input"]
    return original_df
    

In [5]:

metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
column_order = ["instance"]
for metric in metrics:
    column_order.append("{}_Before".format(metric))
    column_order.append("{}_After".format(metric))
    column_order.append("{}_Delta".format(metric))
pd.options.mode.chained_assignment = None 

def get_processed_combined_df(eval_results_path_before, eval_results_path_after, remove_low_iou=True):
    results_df_before_raw = pd.read_pickle(eval_results_path_before)
    results_df_after_raw = pd.read_pickle(eval_results_path_after)

    results_df_before, results_df_after = filter_and_sort_eval_dfs(results_df_before_raw, results_df_after_raw)
    results_df_before = results_df_before.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.reset_index(drop=True)
    results_df_before = results_df_before.reset_index(drop=True)
    results_df_before["CD"] = results_df_before["CD"] * 10
    results_df_after["CD"] = results_df_after["CD"] * 10
    if remove_low_iou:
        results_df_before["3D_IoU"] = results_df_before["3D_IoU"].apply(lambda x: (np.nan if x < 0.01 else x))
    results_df_after["3D_IoU"][np.isnan(results_df_before["3D_IoU"])] = np.nan
    
    metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
    delta_df = results_df_after[metrics] / results_df_before[metrics]
    delta_df = delta_df.replace([np.inf, -np.inf], np.nan)
    
    print("Note: filtering results from {} to {}\n".format(max(len(results_df_before_raw), len(results_df_after_raw)), len(results_df_after)))
    
    results_df_before = results_df_before.rename(columns={metric:"{}_Before".format(metric) for metric in metrics})
    results_df_before = results_df_before[["instance"]+["{}_Before".format(metric) for metric in metrics]]
    results_df_after = results_df_after.rename(columns={metric:"{}_After".format(metric)  for metric in metrics})
    results_df_after = results_df_after[["{}_After".format(metric) for metric in metrics]]
    delta_df = delta_df.rename(columns={metric:"{}_Delta".format(metric) for metric in metrics})
    delta_df = delta_df[["{}_Delta".format(metric) for metric in metrics]]
    
    combined_df = pd.concat([results_df_before, results_df_after, delta_df], axis=1)[column_order]
    
    return combined_df

def make_df_readable(original_df):
    readable_df = original_df.copy()
    readable_df.loc['mean'] = readable_df.mean()
    readable_df = readable_df.round(2)
    for column_name in original_df.columns.to_list():
        if ("_D" in column_name) or ("_C" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "+{}%".format(round((x-1)*100)) if (x-1) > 0 else  "{}%".format(round((x-1)*100)))
        elif ("_P_" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "{}%".format(round((x)*100)))
    return readable_df

def print_latex(input_df):
    for index, row in input_df.iterrows():
        latex_exp = " & ".join([str(x) for x in row.to_list()])
        latex_exp += " \\\\"
        latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
        print(latex_exp)

In [9]:
# shapenet
shapenet_class_ids = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
shapenet_id_to_name = {"02691156":"Airplane", "02828884":"Bench", "02933112":"Cabinet", "02958343":"Car", "03001627":"Chair", "03211117":"Display", "03636649":"Lamp",
                       "03691459":"Speakers", "04090263":"Rifle", "04256520":"Sofa", "04379243":"Table", "04401088":"Telephone", "04530566":"Watercraft"}
pix3d_class_ids = ["bed", "bookcase", "chair", "desk", "misc", "sofa", "table","tool", "wardrobe"]
pix3d_id_to_name = {pix3d_id:pix3d_id.capitalize() for pix3d_id in pix3d_class_ids}

method_paths_dict = {
    "atlasnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/AtlasNet/data/pytorch_3d_render_recs/{}",
        "eval_dir_temp_after": "data/refinements/atlasnet/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "pix2mesh":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/{}/rgba",
        "eval_dir_temp_after": "data/refinements/pix2mesh/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "occnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/occupancy_networks/out/pytorch3d_renders/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/shapenet_occnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose", "bf_pose", "ablation_1", "ablation_2", "ablation_3", "ablation_4"]
    },
    "occnet_pix3d":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/occupancy_networks/out/pix3d/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/pix3d_occnet_refinements/{}/{}",
        "class_ids": pix3d_class_ids,
        "name_cvt_dict": pix3d_id_to_name,
        "available_settings":["gt_pose"]
    },
    "occnet_asym":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/occupancy_networks/out/asym_pytorch3d_renders/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/asym_shapenet_occnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["ablation_2", "ablation_3"]
    },
}

In [7]:
# view specific class

method = "occnet_pix3d"
#method = "occnet"
setting = "gt_pose"
class_id = "chair"

eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, True)

# filtering
#instances = ["d2f8a99bbdc387c8c5552bebbfa48bd7", "dd4da4309c12d47bc2c2c81e2232aa95", "fc7c3ccb57f65337209009cfb89d4bd"]
#combined_df = combined_df[combined_df["instance"].isin(instances)]

#display(combined_df)

print("most improved: {}".format(combined_df.sort_values("MV_IoU_Delta", ascending=False)["instance"].to_list()[:20]))
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(combined_df.sort_values("MV_IoU_Delta", ascending=False))
display(combined_df.sort_values("MV_IoU_Delta", ascending=False))

FileNotFoundError: [Errno 2] No such file or directory: '/home/svcl-oowl/brandon/research/occupancy_networks/out/pix3d/chair/generation/meshes/eval_results.pkl'

In [9]:
# just show single table
method = "occnet"
setting = "gt_pose"
class_ids = method_paths_dict[method]["class_ids"]

quant_results_df_mean = pd.DataFrame()
for class_id in class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = method_paths_dict[method]["name_cvt_dict"][class_id]
    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
display(readable_quant_results_df_mean)

,instance,2D_IoU_Before,2D_IoU_After,2D_IoU_Delta,MV_IoU_Before,MV_IoU_After,MV_IoU_Delta,3D_IoU_Before,3D_IoU_After,3D_IoU_Delta,CD_Before,CD_After,CD_Delta
0,Airplane,0.46,0.64,+61%,0.48,0.55,+18%,0.39,0.41,+20%,0.08,0.03,-37%
1,Bench,0.65,0.76,+24%,0.57,0.60,+8%,0.21,0.21,+18%,0.10,0.06,-6%
2,Cabinet,0.80,0.86,+8%,0.80,0.82,+2%,0.46,0.47,+3%,0.04,0.04,-11%
3,Car,0.80,0.91,+19%,0.79,0.83,+7%,0.30,0.31,+11%,0.05,0.03,-10%
4,Chair,0.62,0.69,+16%,0.56,0.59,+6%,0.32,0.35,+24%,0.14,0.10,-17%
5,Display,0.74,0.83,+14%,0.71,0.75,+6%,0.32,0.38,+59%,0.09,0.06,-28%
6,Lamp,0.57,0.69,+46%,0.55,0.60,+16%,0.26,0.27,+19%,0.45,0.29,-14%
7,Speakers,0.77,0.83,+11%,0.76,0.78,+4%,0.45,0.46,+9%,0.07,0.05,-10%
8,Rifle,0.55,0.68,+80%,0.51,0.55,+12%,0.31,0.30,+17%,0.15,0.05,-27%
9,Sofa,0.78,0.85,+14%,0.75,0.77,+4%,0.49,0.50,+10%,0.06,0.04,+12%


# Tables

In [12]:
# Table 1: Ablation study, Shapenet, occnet, different losses

method = "occnet"
settings = ["ablation_1", "ablation_2", "ablation_3", "gt_pose", "ablation_4"]

ablation_df = pd.DataFrame()
for setting in settings:
    print(setting)
    quant_results_df_mean = pd.DataFrame()
    for class_id in shapenet_class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, True)
        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)
    
    df_order = ["instance"]+["{}_Delta".format(metric) for metric in metrics]
    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)[df_order]
    
    display(readable_quant_results_df_mean)
    ablation_mean_dict = readable_quant_results_df_mean.loc["mean"].to_dict()
    del ablation_mean_dict["instance"]
    ablation_mean_dict["ablation_setting"] = setting
    ablation_df = ablation_df.append(ablation_mean_dict, ignore_index=True)

df_order = ["ablation_setting"]+["{}_Delta".format(metric) for metric in metrics]
ablation_df = ablation_df[df_order]
display(ablation_df)
print_latex(ablation_df)


ablation_1


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+64%,-21%,-80%,+2555%
1,Bench,+32%,-5%,-57%,+867%
2,Cabinet,+10%,-6%,-25%,+288%
3,Car,+21%,-5%,-26%,+333%
4,Chair,+24%,-3%,-44%,+542%
5,Display,+16%,-7%,-28%,+416%
6,Lamp,+48%,-5%,-40%,+1153%
7,Speakers,+13%,-5%,-28%,+517%
8,Rifle,+29%,-35%,-82%,+10261%
9,Sofa,+18%,-5%,-31%,+339%


ablation_2


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+64%,+12%,-5%,-4%
1,Bench,+25%,+6%,-2%,+12%
2,Cabinet,+8%,+1%,-1%,+5%
3,Car,+19%,+4%,+5%,+4%
4,Chair,+20%,+3%,+12%,-6%
5,Display,+14%,+2%,+33%,-10%
6,Lamp,+55%,+11%,+4%,+6%
7,Speakers,+11%,+2%,+5%,+8%
8,Rifle,+74%,-6%,-13%,+129%
9,Sofa,+15%,+3%,+5%,+2%


ablation_3


...

,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+61%,+18%,+20%,-37%
1,Bench,+24%,+8%,+18%,-6%
2,Cabinet,+8%,+2%,+3%,-11%
3,Car,+19%,+7%,+11%,-10%
4,Chair,+16%,+6%,+24%,-17%
5,Display,+14%,+6%,+59%,-28%
6,Lamp,+46%,+16%,+19%,-14%
7,Speakers,+11%,+4%,+9%,-10%
8,Rifle,+80%,+12%,+17%,-27%
9,Sofa,+14%,+4%,+10%,+12%


ablation_4


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+74%,+23%,+7%,-48%
1,Bench,+31%,+13%,+12%,-9%
2,Cabinet,+8%,+2%,+5%,-4%
3,Car,+22%,+9%,+13%,-13%
4,Chair,+28%,+11%,+46%,-24%
5,Display,+15%,+6%,+33%,-20%
6,Lamp,+64%,+23%,+15%,-18%
7,Speakers,+11%,+4%,+13%,-6%
8,Rifle,+91%,+18%,+14%,-47%
9,Sofa,+13%,+3%,+5%,+89%


,ablation_setting,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,ablation_1,+27%,-9%,-45%,+1628%
1,ablation_2,+29%,+4%,+5%,+16%
2,ablation_3,+31%,+4%,+6%,+12%
3,gt_pose,+28%,+8%,+24%,-15%
4,ablation_4,+35%,+11%,+20%,-11%


ablation_1 & +27\% & -9\% & -45\% & +1628\% \\
ablation_2 & +29\% & +4\% & +5\% & +16\% \\
ablation_3 & +31\% & +4\% & +6\% & +12\% \\
gt_pose & +28\% & +8\% & +24\% & -15\% \\
ablation_4 & +35\% & +11\% & +20\% & -11\% \\


In [9]:
# Table 1, last 2 rows: asym ablation

method = "occnet_asym"
#settings = ["ablation_1"]
#settings = ["ablation_1", "ablation_2", "ablation_3"]
settings = ["ablation_2", "ablation_3"]

ablation_df = pd.DataFrame()
for setting in settings:
    print(setting)
    quant_results_df_mean = pd.DataFrame()
    for class_id in shapenet_class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, True)
        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)
    
    df_order = ["instance"]+["{}_Delta".format(metric) for metric in metrics]
    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)[df_order]
    
    display(readable_quant_results_df_mean)
    ablation_mean_dict = readable_quant_results_df_mean.loc["mean"].to_dict()
    del ablation_mean_dict["instance"]
    ablation_mean_dict["ablation_setting"] = setting
    ablation_df = ablation_df.append(ablation_mean_dict, ignore_index=True)

df_order = ["ablation_setting"]+["{}_Delta".format(metric) for metric in metrics]
ablation_df = ablation_df[df_order]
display(ablation_df)
print_latex(ablation_df)


ablation_2


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+83%,+8%,+3%,-16%
1,Bench,+68%,+19%,+23%,+19%
2,Cabinet,+36%,-1%,+10%,+1%
3,Car,+43%,+15%,+14%,-13%
4,Chair,+16%,+5%,0%,+26%
5,Display,+13%,+6%,+324%,-26%
6,Lamp,+53%,+14%,+48%,-22%
7,Speakers,+19%,+6%,+9%,-3%
8,Rifle,+75%,+42%,-3%,-1%
9,Sofa,+55%,+12%,+28%,+156%


ablation_3


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+118%,+14%,+3%,-14%
1,Bench,+100%,+26%,-2%,-23%
2,Cabinet,+43%,+2%,+59%,+4%
3,Car,+34%,+16%,+21%,-6%
4,Chair,+27%,+8%,-7%,+29%
5,Display,+18%,+5%,+604%,-38%
6,Lamp,+61%,+16%,+50%,-21%
7,Speakers,+19%,+4%,+6%,+12%
8,Rifle,+90%,+46%,-7%,+2%
9,Sofa,+56%,+14%,+48%,+35%


,ablation_setting,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,ablation_2,+63%,+14%,+45%,+9%
1,ablation_3,+75%,+16%,+68%,-1%


ablation_2 & +63\% & +14\% & +45\% & +9\% \\
ablation_3 & +75\% & +16\% & +68\% & -1\% \\


In [20]:
# table 2: all three methods side by side on shapenet with gt pose, only percentage change info
methods = ["occnet", "atlasnet", "pix2mesh"]

dfs_to_combine = []
for method in methods:
    class_ids = method_paths_dict[method]["class_ids"]
    quant_results_df_mean = pd.DataFrame()
    setting = "gt_pose"
    for class_id in class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
    if len(dfs_to_combine) == 0:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
    else:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["{}_Delta".format(metric) for metric in metrics]]
    readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    dfs_to_combine.append(readable_quant_results_df_mean)
    
df_order = ["instance"]+["{}_{}".format(method, metric) for method in methods  for metric in metrics]
methods_combined_df = pd.concat(dfs_to_combine, axis=1)[df_order]
display(methods_combined_df)
print_latex(methods_combined_df)

Note: filtering results from 809 to 790

Note: filtering results from 363 to 341

Note: filtering results from 190 to 185

Note: filtering results from 1499 to 1496

Note: filtering results from 1355 to 1300

Note: filtering results from 219 to 212

Note: filtering results from 463 to 361

Note: filtering results from 323 to 309

Note: filtering results from 474 to 437

Note: filtering results from 634 to 612

Note: filtering results from 1701 to 1642

Note: filtering results from 210 to 200

Note: filtering results from 387 to 359

Note: filtering results from 809 to 794

Note: filtering results from 363 to 359

Note: filtering results from 190 to 185

Note: filtering results from 1499 to 1496

Note: filtering results from 1355 to 1309

Note: filtering results from 219 to 212

Note: filtering results from 463 to 384

Note: filtering results from 323 to 309

Note: filtering results from 474 to 460

Note: filtering results from 634 to 612

Note: filtering results from 1701 to 1653

Note

,instance,occnet_2D_IoU,occnet_MV_IoU,occnet_3D_IoU,occnet_CD,atlasnet_2D_IoU,atlasnet_MV_IoU,atlasnet_3D_IoU,atlasnet_CD,pix2mesh_2D_IoU,pix2mesh_MV_IoU,pix2mesh_3D_IoU,pix2mesh_CD
0,Airplane,+61%,+18%,+20%,-37%,+51%,+25%,+130%,-61%,+61%,+25%,+177%,-48%
1,Bench,+24%,+8%,+18%,-6%,+31%,+8%,+46%,-26%,+18%,+6%,+61%,-26%
2,Cabinet,+8%,+2%,+3%,-11%,+16%,+7%,+18%,-34%,+10%,+5%,+5%,-18%
3,Car,+19%,+7%,+11%,-10%,+54%,+27%,+75%,-74%,+9%,+7%,+18%,-27%
4,Chair,+16%,+6%,+24%,-17%,+16%,+9%,+57%,-45%,+11%,+8%,+67%,-30%
5,Display,+14%,+6%,+59%,-28%,+16%,+6%,+40%,-28%,+12%,+9%,+44%,-33%
6,Lamp,+46%,+16%,+19%,-14%,+44%,+17%,+103%,-46%,+18%,+7%,+97%,-35%
7,Speakers,+11%,+4%,+9%,-10%,+16%,+7%,+21%,-35%,+8%,+5%,+9%,-16%
8,Rifle,+80%,+12%,+17%,-27%,+68%,+34%,+128%,-65%,+25%,+29%,+117%,-51%
9,Sofa,+14%,+4%,+10%,+12%,+20%,+6%,+22%,-36%,+11%,+3%,+28%,-20%


Airplane & +61\% & +18\% & +20\% & -37\% & +51\% & +25\% & +130\% & -61\% & +61\% & +25\% & +177\% & -48\% \\
Bench & +24\% & +8\% & +18\% & -6\% & +31\% & +8\% & +46\% & -26\% & +18\% & +6\% & +61\% & -26\% \\
Cabinet & +8\% & +2\% & +3\% & -11\% & +16\% & +7\% & +18\% & -34\% & +10\% & +5\% & +5\% & -18\% \\
Car & +19\% & +7\% & +11\% & -10\% & +54\% & +27\% & +75\% & -74\% & +9\% & +7\% & +18\% & -27\% \\
Chair & +16\% & +6\% & +24\% & -17\% & +16\% & +9\% & +57\% & -45\% & +11\% & +8\% & +67\% & -30\% \\
Display & +14\% & +6\% & +59\% & -28\% & +16\% & +6\% & +40\% & -28\% & +12\% & +9\% & +44\% & -33\% \\
Lamp & +46\% & +16\% & +19\% & -14\% & +44\% & +17\% & +103\% & -46\% & +18\% & +7\% & +97\% & -35\% \\
Speakers & +11\% & +4\% & +9\% & -10\% & +16\% & +7\% & +21\% & -35\% & +8\% & +5\% & +9\% & -16\% \\
Rifle & +80\% & +12\% & +17\% & -27\% & +68\% & +34\% & +128\% & -65\% & +25\% & +29\% & +117\% & -51\% \\
Sofa & +14\% & +4\% & +10\% & +12\% & +20\% & +6\% & +22\% & -36\% & 

In [9]:
# table 3: occnet on pix3d with gt pose, only percentage change info

method = "occnet_pix3d"
quant_results_df_mean = pd.DataFrame()
setting = "gt_pose"
for class_id in pix3d_class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = class_id.capitalize()
    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    
display(readable_quant_results_df_mean)
print_latex(readable_quant_results_df_mean)

,instance,occnet_pix3d_2D_IoU,occnet_pix3d_MV_IoU,occnet_pix3d_3D_IoU,occnet_pix3d_CD
0,Bed,+57%,+18%,+23%,-17%
1,Bookcase,+75%,+28%,+3%,-29%
2,Chair,+105%,+17%,+23%,-15%
3,Desk,+73%,+10%,+8%,-3%
4,Misc,+120%,+40%,+23%,-31%
5,Sofa,+39%,+8%,+8%,+7%
6,Table,+45%,+11%,+24%,+2%
7,Tool,+94%,+19%,+31%,-32%
8,Wardrobe,+52%,+14%,+7%,+4%
mean,NaN,+73%,+18%,+17%,-13%


Bed & +57\% & +18\% & +23\% & -17\% \\
Bookcase & +75\% & +28\% & +3\% & -29\% \\
Chair & +105\% & +17\% & +23\% & -15\% \\
Desk & +73\% & +10\% & +8\% & -3\% \\
Misc & +120\% & +40\% & +23\% & -31\% \\
Sofa & +39\% & +8\% & +8\% & +7\% \\
Table & +45\% & +11\% & +24\% & +2\% \\
Tool & +94\% & +19\% & +31\% & -32\% \\
Wardrobe & +52\% & +14\% & +7\% & +4\% \\
Mean & +73\% & +18\% & +17\% & -13\% \\


In [10]:
# REMOVED TABLE: occnet on shapenet bf pose, also showing percentage/amt up

method = "occnet"
setting = "bf_pose"
quant_results_df_mean = pd.DataFrame()
class_ids = method_paths_dict[method]["class_ids"]

for class_id in class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, False)
    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]

    for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
        if metric in ["CD"]:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
        else:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()

        quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
        quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt


    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
df_order = ["instance"]+["{}_{}".format(metric, i) for metric in ["MV_IoU", "3D_IoU", "CD"] for i in ["P_up", "C_up", "C"]]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(readable_quant_results_df_mean[df_order])

print_latex(readable_quant_results_df_mean[df_order])


Note: filtering results from 809 to 791

Note: filtering results from 363 to 340

Note: filtering results from 190 to 185

Note: filtering results from 1499 to 1496

Note: filtering results from 1355 to 1301

Note: filtering results from 219 to 212

Note: filtering results from 463 to 361

Note: filtering results from 323 to 309

Note: filtering results from 474 to 439

Note: filtering results from 634 to 612

Note: filtering results from 1701 to 1642

Note: filtering results from 210 to 200

Note: filtering results from 387 to 358



,instance,MV_IoU_P_up,MV_IoU_C_up,MV_IoU_C,3D_IoU_P_up,3D_IoU_C_up,3D_IoU_C,CD_P_up,CD_C_up,CD_C
0,Airplane,41%,+12%,-3%,22%,+190%,+15%,34%,-30%,+76%
1,Bench,39%,+8%,-1%,33%,+66%,-6%,27%,-20%,+69%
2,Cabinet,35%,+2%,-3%,24%,+26%,-3%,25%,-14%,+65%
3,Car,41%,+4%,-1%,39%,+13%,-2%,37%,-14%,+27%
4,Chair,60%,+5%,+2%,44%,+132%,+45%,55%,-24%,+6%
5,Display,27%,+4%,-7%,35%,+325%,+96%,29%,-24%,+98%
6,Lamp,59%,+14%,+5%,43%,+98%,+26%,54%,-23%,+13%
7,Speakers,46%,+3%,-1%,35%,+25%,0%,30%,-16%,+45%
8,Rifle,19%,+20%,-15%,17%,+90%,-19%,29%,-38%,+202%
9,Sofa,34%,+4%,-3%,29%,+17%,-8%,27%,-16%,+81%


Airplane & 41\% & +12\% & -3\% & 22\% & +190\% & +15\% & 34\% & -30\% & +76\% \\
Bench & 39\% & +8\% & -1\% & 33\% & +66\% & -6\% & 27\% & -20\% & +69\% \\
Cabinet & 35\% & +2\% & -3\% & 24\% & +26\% & -3\% & 25\% & -14\% & +65\% \\
Car & 41\% & +4\% & -1\% & 39\% & +13\% & -2\% & 37\% & -14\% & +27\% \\
Chair & 60\% & +5\% & +2\% & 44\% & +132\% & +45\% & 55\% & -24\% & +6\% \\
Display & 27\% & +4\% & -7\% & 35\% & +325\% & +96\% & 29\% & -24\% & +98\% \\
Lamp & 59\% & +14\% & +5\% & 43\% & +98\% & +26\% & 54\% & -23\% & +13\% \\
Speakers & 46\% & +3\% & -1\% & 35\% & +25\% & 0\% & 30\% & -16\% & +45\% \\
Rifle & 19\% & +20\% & -15\% & 17\% & +90\% & -19\% & 29\% & -38\% & +202\% \\
Sofa & 34\% & +4\% & -3\% & 29\% & +17\% & -8\% & 27\% & -16\% & +81\% \\
Table & 47\% & +8\% & +1\% & 46\% & +281\% & +109\% & 40\% & -21\% & +25\% \\
Telephone & 16\% & +4\% & -13\% & 16\% & +119\% & -13\% & 16\% & -21\% & +696\% \\
Watercraft & 45\% & +9\% & -2\% & 34\% & +71\% & +6\% & 30\% & -22\% & +

In [17]:
# appendix raw tables (TODO: make sure of iou filter consistency)

for method in method_paths_dict:
    
    for setting in method_paths_dict[method]["available_settings"]:
        print("\n ============================================================================================================== \n")
        print("{} -- {}".format(method, setting))
        quant_results_df_mean = pd.DataFrame()
        
        for class_id in method_paths_dict[method]["class_ids"]:
            eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
            eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

            combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)
            quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
            quant_result_dict_mean["instance"] = method_paths_dict[method]["name_cvt_dict"][class_id]

            for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
                if metric in ["CD"]:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                else:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean(skipna=True)
                    
                if num_improved == 0:
                    avg_improve_amt = 0.0
                if num_worse == 0:
                    avg_worse_amt = 0.0
                        
                quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
                quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt

            quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
        readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
        df_order = ["instance"]+["{}_{}".format(metric, i) for metric in metrics for i in ["B", "A", "C", "P_up", "C_up", "P_d", "C_d"]]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(readable_quant_results_df_mean[df_order].transpose())
        print_latex_full(readable_quant_results_df_mean, df_order, method)
        #print_latex(readable_quant_results_df_mean[df_order].transpose())




atlasnet -- gt_pose


,0,1,2,3,4,5,6,7,8,9,10,11,12,mean
instance,Airplane,Bench,Cabinet,Car,Chair,Display,Lamp,Speakers,Rifle,Sofa,Table,Telephone,Watercraft,NaN
2D_IoU_B,0.38,0.58,0.71,0.52,0.54,0.7,0.47,0.71,0.37,0.69,0.6,0.7,0.47,0.57
2D_IoU_A,0.56,0.73,0.81,0.76,0.63,0.8,0.63,0.81,0.56,0.8,0.76,0.82,0.69,0.72
2D_IoU_C,+51%,+31%,+16%,+54%,+16%,+16%,+44%,+16%,+68%,+20%,+30%,+19%,+62%,+34%
2D_IoU_P_up,96%,93%,94%,99%,88%,96%,93%,92%,97%,93%,96%,96%,97%,95%
2D_IoU_C_up,+53%,+33%,+17%,+54%,+19%,+17%,+48%,+18%,+71%,+22%,+32%,+20%,+64%,+36%
2D_IoU_P_d,4%,7%,6%,1%,12%,4%,7%,8%,3%,7%,4%,4%,3%,5%
2D_IoU_C_d,-12%,-9%,-3%,-3%,-8%,-7%,-19%,-3%,-18%,-3%,-5%,-17%,-9%,-9%
MV_IoU_B,0.36,0.5,0.69,0.5,0.5,0.63,0.45,0.7,0.33,0.65,0.53,0.63,0.46,0.53
MV_IoU_A,0.45,0.53,0.73,0.63,0.54,0.67,0.52,0.74,0.44,0.69,0.57,0.69,0.56,0.6


\hline
 & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
\cline{1-16}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.38 & 0.58 & 0.71 & 0.52 & 0.54 & 0.7 & 0.47 & 0.71 & 0.37 & 0.69 & 0.6 & 0.7 & 0.47 & 0.57 \\
  & After & 0.56 & 0.73 & 0.81 & 0.76 & 0.63 & 0.8 & 0.63 & 0.81 & 0.56 & 0.8 & 0.76 & 0.82 & 0.69 & 0.72 \\
  & P.C. & +51\% & +31\% & +16\% & +54\% & +16\% & +16\% & +44\% & +16\% & +68\% & +20\% & +30\% & +19\% & +62\% & +34\% \\
\cline{2-16}
  & Prop. $\uparrow$ & 96\% & 93\% & 94\% & 99\% & 88\% & 96\% & 93\% & 92\% & 97\% & 93\% & 96\% & 96\% & 97\% & 95\% \\
  & Amt.  $\uparrow$ & +53\% & +33\% & +17\% & +54\% & +19\% & +17\% & +48\% & +18\% & +71\% & +22\% & +32\% & +20\% & +64\% & +36\% \\
\cline{2-16}
  & Prop.  $\downarrow$ & 4\% & 7\% & 6\% & 1\% & 12\% & 4\% & 7\% & 8\% & 3\% & 7\% & 4\% & 4\% & 3\% & 5\% \\
  & Amt. $\downarrow$ & -12\% & -9\% & -3\% &

,0,1,2,3,4,5,6,7,8,9,10,11,12,mean
instance,Airplane,Bench,Cabinet,Car,Chair,Display,Lamp,Speakers,Rifle,Sofa,Table,Telephone,Watercraft,NaN
2D_IoU_B,0.45,0.7,0.84,0.87,0.7,0.81,0.63,0.84,0.6,0.81,0.69,0.84,0.73,0.73
2D_IoU_A,0.66,0.82,0.92,0.94,0.77,0.89,0.72,0.9,0.73,0.9,0.8,0.92,0.86,0.83
2D_IoU_C,+61%,+18%,+10%,+9%,+11%,+12%,+18%,+8%,+25%,+11%,+17%,+10%,+21%,+18%
2D_IoU_P_up,97%,96%,91%,98%,86%,93%,82%,90%,93%,92%,88%,96%,94%,92%
2D_IoU_C_up,+63%,+19%,+11%,+9%,+14%,+13%,+26%,+9%,+28%,+12%,+20%,+11%,+23%,+20%
2D_IoU_P_d,3%,4%,9%,2%,14%,7%,18%,10%,7%,8%,12%,4%,6%,8%
2D_IoU_C_d,-14%,-7%,-2%,-2%,-9%,-4%,-20%,-3%,-18%,-6%,-8%,-28%,-13%,-10%
MV_IoU_B,0.45,0.6,0.79,0.78,0.59,0.71,0.55,0.77,0.44,0.76,0.62,0.74,0.6,0.65
MV_IoU_A,0.56,0.63,0.83,0.83,0.64,0.77,0.58,0.8,0.55,0.78,0.63,0.8,0.69,0.7


\hline
 & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
\cline{1-16}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.45 & 0.7 & 0.84 & 0.87 & 0.7 & 0.81 & 0.63 & 0.84 & 0.6 & 0.81 & 0.69 & 0.84 & 0.73 & 0.73 \\
  & After & 0.66 & 0.82 & 0.92 & 0.94 & 0.77 & 0.89 & 0.72 & 0.9 & 0.73 & 0.9 & 0.8 & 0.92 & 0.86 & 0.83 \\
  & P.C. & +61\% & +18\% & +10\% & +9\% & +11\% & +12\% & +18\% & +8\% & +25\% & +11\% & +17\% & +10\% & +21\% & +18\% \\
\cline{2-16}
  & Prop. $\uparrow$ & 97\% & 96\% & 91\% & 98\% & 86\% & 93\% & 82\% & 90\% & 93\% & 92\% & 88\% & 96\% & 94\% & 92\% \\
  & Amt.  $\uparrow$ & +63\% & +19\% & +11\% & +9\% & +14\% & +13\% & +26\% & +9\% & +28\% & +12\% & +20\% & +11\% & +23\% & +20\% \\
\cline{2-16}
  & Prop.  $\downarrow$ & 3\% & 4\% & 9\% & 2\% & 14\% & 7\% & 18\% & 10\% & 7\% & 8\% & 12\% & 4\% & 6\% & 8\% \\
  & Amt. $\downarrow$ & -14\% & -7\% & -2\% & -

...

\hline
 & & Bed & Bookcase & Chair & Desk & Misc & Sofa & Table & Tool & Wardrobe & Mean \\
\cline{1-12}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.59 & 0.65 & 0.53 & 0.56 & 0.55 & 0.76 & 0.56 & 0.48 & 0.8 & 0.61 \\
  & After & 0.79 & 0.84 & 0.65 & 0.71 & 0.8 & 0.89 & 0.68 & 0.8 & 0.94 & 0.79 \\
  & P.C. & +57\% & +75\% & +105\% & +73\% & +120\% & +39\% & +45\% & +94\% & +52\% & +73\% \\
\cline{2-12}
  & Prop. $\uparrow$ & 95\% & 95\% & 87\% & 83\% & 95\% & 90\% & 79\% & 93\% & 95\% & 90\% \\
  & Amt.  $\uparrow$ & +60\% & +79\% & +121\% & +90\% & +127\% & +43\% & +59\% & +102\% & +55\% & +82\% \\
\cline{2-12}
  & Prop.  $\downarrow$ & 5\% & 5\% & 13\% & 17\% & 5\% & 10\% & 21\% & 7\% & 5\% & 10\% \\
  & Amt. $\downarrow$ & -3\% & -3\% & -6\% & -9\% & -8\% & -3\% & -9\% & -10\% & -2\% & -6\% \\
\cline{1-12}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{MV IoU}}} & Before & 0.56 & 0.58 & 0.47 & 0.59 & 0.52 & 0.73 & 0.56 & 0.4 & 0.73 & 0.57

,0,1,2,3,4,5,6,7,8,9,10,11,12,mean
instance,Airplane,Bench,Cabinet,Car,Chair,Display,Lamp,Speakers,Rifle,Sofa,Table,Telephone,Watercraft,NaN
2D_IoU_B,0.34,0.4,0.6,0.55,0.51,0.66,0.4,0.69,0.48,0.49,0.49,0.54,0.4,0.5
2D_IoU_A,0.5,0.54,0.79,0.71,0.57,0.73,0.55,0.8,0.63,0.69,0.61,0.73,0.6,0.65
2D_IoU_C,+83%,+68%,+36%,+43%,+16%,+13%,+53%,+19%,+75%,+55%,+35%,+42%,+277%,+63%
2D_IoU_P_up,89%,88%,100%,93%,78%,67%,89%,95%,100%,85%,89%,93%,87%,89%
2D_IoU_C_up,+103%,+81%,+36%,+48%,+22%,+21%,+61%,+20%,+75%,+66%,+45%,+46%,+321%,+73%
2D_IoU_P_d,11%,12%,0%,7%,22%,33%,11%,5%,0%,15%,11%,7%,13%,11%
2D_IoU_C_d,-71%,-30%,-100%,-14%,-4%,-4%,-16%,-4%,-100%,-8%,-47%,-4%,-6%,-31%
MV_IoU_B,0.35,0.39,0.61,0.56,0.45,0.59,0.31,0.64,0.4,0.51,0.48,0.53,0.41,0.48
MV_IoU_A,0.37,0.43,0.61,0.62,0.47,0.62,0.35,0.67,0.45,0.55,0.51,0.57,0.47,0.51


\hline
 & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
\cline{1-16}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.34 & 0.4 & 0.6 & 0.55 & 0.51 & 0.66 & 0.4 & 0.69 & 0.48 & 0.49 & 0.49 & 0.54 & 0.4 & 0.5 \\
  & After & 0.5 & 0.54 & 0.79 & 0.71 & 0.57 & 0.73 & 0.55 & 0.8 & 0.63 & 0.69 & 0.61 & 0.73 & 0.6 & 0.65 \\
  & P.C. & +83\% & +68\% & +36\% & +43\% & +16\% & +13\% & +53\% & +19\% & +75\% & +55\% & +35\% & +42\% & +277\% & +63\% \\
\cline{2-16}
  & Prop. $\uparrow$ & 89\% & 88\% & 100\% & 93\% & 78\% & 67\% & 89\% & 95\% & 100\% & 85\% & 89\% & 93\% & 87\% & 89\% \\
  & Amt.  $\uparrow$ & +103\% & +81\% & +36\% & +48\% & +22\% & +21\% & +61\% & +20\% & +75\% & +66\% & +45\% & +46\% & +321\% & +73\% \\
\cline{2-16}
  & Prop.  $\downarrow$ & 11\% & 12\% & 0\% & 7\% & 22\% & 33\% & 11\% & 5\% & 0\% & 15\% & 11\% & 7\% & 13\% & 11\% \\
  & Amt. $\downarrow$ & -71\% & -30

,0,1,2,3,4,5,6,7,8,9,10,11,12,mean
instance,Airplane,Bench,Cabinet,Car,Chair,Display,Lamp,Speakers,Rifle,Sofa,Table,Telephone,Watercraft,NaN
2D_IoU_B,0.34,0.4,0.6,0.55,0.51,0.66,0.4,0.69,0.48,0.49,0.49,0.54,0.4,0.5
2D_IoU_A,0.58,0.6,0.83,0.7,0.6,0.77,0.57,0.81,0.65,0.69,0.63,0.78,0.66,0.68
2D_IoU_C,+118%,+100%,+43%,+34%,+27%,+18%,+61%,+19%,+90%,+56%,+38%,+52%,+325%,+75%
2D_IoU_P_up,100%,88%,100%,86%,89%,83%,74%,89%,94%,85%,83%,93%,93%,89%
2D_IoU_C_up,+118%,+115%,+43%,+41%,+30%,+23%,+87%,+23%,+97%,+67%,+51%,+56%,+348%,+85%
2D_IoU_P_d,0%,12%,0%,14%,11%,17%,26%,11%,6%,15%,17%,7%,7%,11%
2D_IoU_C_d,-100%,-14%,-100%,-9%,-2%,-10%,-10%,-15%,-17%,-7%,-22%,-4%,-5%,-24%
MV_IoU_B,0.35,0.39,0.61,0.56,0.45,0.59,0.31,0.64,0.4,0.51,0.48,0.53,0.41,0.48
MV_IoU_A,0.38,0.45,0.62,0.62,0.48,0.62,0.35,0.67,0.46,0.56,0.5,0.56,0.5,0.52


\hline
 & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\
\cline{1-16}
\parbox[t]{3mm}{\multirow{7}{*}{\rotatebox[origin=c]{90}{2D IoU}}} & Before & 0.34 & 0.4 & 0.6 & 0.55 & 0.51 & 0.66 & 0.4 & 0.69 & 0.48 & 0.49 & 0.49 & 0.54 & 0.4 & 0.5 \\
  & After & 0.58 & 0.6 & 0.83 & 0.7 & 0.6 & 0.77 & 0.57 & 0.81 & 0.65 & 0.69 & 0.63 & 0.78 & 0.66 & 0.68 \\
  & P.C. & +118\% & +100\% & +43\% & +34\% & +27\% & +18\% & +61\% & +19\% & +90\% & +56\% & +38\% & +52\% & +325\% & +75\% \\
\cline{2-16}
  & Prop. $\uparrow$ & 100\% & 88\% & 100\% & 86\% & 89\% & 83\% & 74\% & 89\% & 94\% & 85\% & 83\% & 93\% & 93\% & 89\% \\
  & Amt.  $\uparrow$ & +118\% & +115\% & +43\% & +41\% & +30\% & +23\% & +87\% & +23\% & +97\% & +67\% & +51\% & +56\% & +348\% & +85\% \\
\cline{2-16}
  & Prop.  $\downarrow$ & 0\% & 12\% & 0\% & 14\% & 11\% & 17\% & 26\% & 11\% & 6\% & 15\% & 17\% & 7\% & 7\% & 11\% \\
  & Amt. $\downarrow$ & -100\% &

In [16]:
# occnet_pix3d

def print_latex_full(readable_quant_results_df_mean, df_order, method):
    i = 0
    first = True
    print("\\hline")
    if method == "occnet_pix3d":
        print(" & & Bed & Bookcase & Chair & Desk & Misc & Sofa & Table & Tool & Wardrobe & Mean \\\\")
        num_cols = 12
    else:
        print(" & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\\\")
        num_cols = 16
    for index, row in readable_quant_results_df_mean[df_order].transpose().iterrows():
        if first:
            first = False
            continue
        row_list = [str(x) for x in row.to_list()]
        if i%7==0: row_list.insert(0,"Before")
        elif i%7==1: row_list.insert(0,"After")
        elif i%7==2: row_list.insert(0,"P.C.")
        elif i%7==3: row_list.insert(0,"Prop. $\\uparrow$")
        elif i%7==4: row_list.insert(0,"Amt.  $\\uparrow$")
        elif i%7==5: row_list.insert(0,"Prop.  $\\downarrow$")
        elif i%7==6: row_list.insert(0,"Amt. $\\downarrow$")

        if i==0:
            row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{2D IoU}}}")
        elif i==7:
            row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{MV IoU}}}")
        elif i==14:
            row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{3D IoU}}}")
        elif i==21:
            row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{CD}}}")
        else:
            row_list.insert(0," ")

        latex_exp = " & ".join(row_list)
        latex_exp += " \\\\"

        latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
        if i % 7 == 0:
            print("\cline{1-"+str(num_cols) +"}")
        elif(i%7==3 or i%7==5):
            print("\cline{2-"+str(num_cols) +"}")
        print(latex_exp)
        i+=1
    print("\\hline")